In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
sns.set_style('darkgrid')


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
text = pd.read_excel('Orginial_Data.xlsx')


In [4]:
text


,Public_Opinion_Vaccine
0,Making appointments
1,If we were allowed to be exempt from classes f...
2,A streamlined clinic that makes getting the va...
3,Email updates about eligibility
4,It being distributed at hours early and later ...
...,...
1687,has effects on fertility
1688,The side effects.
1689,People literally dying
1690,I have allergies so I am afraid I might have a...


In [5]:
text.drop_duplicates(subset = ['Public_Opinion_Vaccine'],keep = 'first',inplace=True)


In [6]:
text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1631 entries, 0 to 1691
Data columns (total 1 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Public_Opinion_Vaccine  1631 non-null   object
dtypes: object(1)
memory usage: 25.5+ KB


In [7]:
text.describe()

,Public_Opinion_Vaccine
count,1631
unique,1631
top,Different things about my health in the long run
freq,1


In [8]:
from transformers import pipeline


In [9]:
#model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
#sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

In [10]:
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))


[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [11]:
#prediction = sentiment_task(" infertile")
#print(prediction)

In [12]:
def sun():
    hi = "hi"
    return hi

In [13]:
#def predict(text):
#    prediction = sentiment_task(text)
#    return prediction
def predictscore(text):
    prediction = sentiment_analysis(text)
    return prediction[0]["score"]
    
def predictlabel(text):
    prediction = sentiment_analysis(text)
    return prediction[0]["label"]

In [14]:
texttest = text.copy()
list(texttest)

['Public_Opinion_Vaccine']

In [15]:
#prediction = sentiment_task("hi")
#print(prediction[0]["score"])

In [16]:
#texttest["prediction"] = texttest["Public_Opinion_Vaccine"].apply(predict)
texttest["label"] = texttest["Public_Opinion_Vaccine"].apply(predictlabel)
texttest["score"] = texttest["Public_Opinion_Vaccine"].apply(predictscore)


texttest["label"]=texttest.prediction[0]["label"]
texttest["score"]=texttest["prediction"][0]["score"]
texttest.head()

In [17]:
texttest

,Public_Opinion_Vaccine,label,score
0,Making appointments,POSITIVE,0.996486
1,If we were allowed to be exempt from classes f...,NEGATIVE,0.997259
2,A streamlined clinic that makes getting the va...,POSITIVE,0.998895
3,Email updates about eligibility,POSITIVE,0.994933
4,It being distributed at hours early and later ...,POSITIVE,0.995604
...,...,...,...
1687,has effects on fertility,NEGATIVE,0.984783
1688,The side effects.,NEGATIVE,0.996639
1689,People literally dying,NEGATIVE,0.995031
1690,I have allergies so I am afraid I might have a...,NEGATIVE,0.998603


In [18]:
texttest.to_excel("Sentiment_Results_score.xlsx") 

# TEXT PRE_PROCESSING 

In [19]:
import string
from nltk.corpus import stopwords



In [20]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

mess = texttest['Public_Opinion_Vaccine']


In [21]:
text_process(mess)

['Making',
 'appointments',
 'allowed',
 'exempt',
 'classes',
 'day',
 'streamlined',
 'clinic',
 'makes',
 'getting',
 'vaccine',
 'simple',
 'situation.Email',
 'updates',
 'eligibility',
 'distributed',
 'hours',
 'early',
 'later',
 '9-5.',
 'Ex',
 '7â€“7Good',
 'Info',
 'go',
 'doNo',
 'harmful',
 'side',
 'effects',
 'infertility,',
 'perfectly',
 'healthy',
 'want',
 'new',
 'conditions',
 'arise',
 'due',
 'vaccine',
 'thatâ€™s',
 'around',
 'less',
 'year',
 'location',
 'close',
 'campus',
 'possiblemore',
 'places',
 'get',
 'vaccineif',
 'tested',
 'housingInvite',
 'emailWeek',
 'schoolAppt',
 'times',
 'receive',
 'itwalkabilityMore',
 'appointment',
 'times',
 'Close',
 'homelocation',
 'parking',
 'Close',
 'distance',
 'campus',
 'site',
 'information',
 'safety',
 'vaccineI',
 'want',
 'date',
 'get',
 'vaccine',
 'proximity',
 'campusIf',
 'Appalachian',
 'state',
 'campus',
 'BooneIf',
 'available',
 'campus',
 'vaccine',
 'given',
 'campus.',
 'told',
 'get',
 'co

# Vectorization

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
Bag_of_words = CountVectorizer(analyzer=text_process).fit(texttest['Public_Opinion_Vaccine'])


In [24]:
# Print total number of vocab words
print(len(Bag_of_words.vocabulary_))

2144


In [25]:
BOW = Bag_of_words.transform(texttest['Public_Opinion_Vaccine'])
print('Shape of Sparse Matrix: ', BOW.shape)
print('Amount of Non-Zero occurences: ', BOW.nnz)

Shape of Sparse Matrix:  (1631, 2144)
Amount of Non-Zero occurences:  8548


In [26]:
sparsity = (100.0 * BOW.nnz / (BOW.shape[0] * BOW.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity: 0


# TF-IDF

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(BOW)
td_idf = tfidf_transformer.transform(BOW)
print(td_idf.shape)

(1631, 2144)


In [51]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.svm import LinearSVC


# Creating a Data Pipeline

In [52]:
X = texttest['Public_Opinion_Vaccine']
y = texttest['label']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.25)

In [54]:
display(X_test)

139     Verifying with my doctor that now is a good time.
376     I might wait until after the school year so th...
1011    *Only* making appointments available on weekda...
249                                         more research
377     Being able to choose an appointment time that ...
                              ...                        
429                       A flexible time to get it done.
1584    I do not trust a rushed vaccine to a virus tha...
123                        if they require us to take it.
179                                 Morning availability 
1506     Negative side effects (IE sickness the next day)
Name: Public_Opinion_Vaccine, Length: 408, dtype: object

In [55]:
X_test[1:2]

376    I might wait until after the school year so th...
Name: Public_Opinion_Vaccine, dtype: object

In [56]:
X_train

1051                                  My school schedule.
578                   prior notice and clear instructions
1513                      Severe Side Effects like fever 
78      Offer it sometime in the afternoon on a weekda...
267                                Orderly Accessibility 
                              ...                        
1224                              the symptoms it causes 
284                          More places to be vaccinated
803           Not enough research or time to back it up. 
1246    Evidence that the vaccine causes terrible side...
443                            fast and efficient process
Name: Public_Opinion_Vaccine, Length: 1223, dtype: object

In [69]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', DecisionTreeClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [70]:
pipe.fit (X_train,y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x000002335A2B7B80>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', DecisionTreeClassifier())])

In [71]:
predictions = pipe.predict(X_test)

In [72]:
print (classification_report(predictions, y_test))

              precision    recall  f1-score   support

    NEGATIVE       0.72      0.67      0.69       253
    POSITIVE       0.52      0.58      0.55       155

    accuracy                           0.63       408
   macro avg       0.62      0.62      0.62       408
weighted avg       0.64      0.63      0.64       408



# Emotion detection 

In [38]:
emotion = pipeline('sentiment-analysis',model= 'bhadresh-savani/distilbert-base-uncased-emotion')
prediction = emotion("I am coming" )
print(prediction)

[{'label': 'joy', 'score': 0.8065890669822693}]


In [39]:
overall_collection = []

for i in texttest['Public_Opinion_Vaccine']:
    collection = emotion(i)
    overall_collection.append(collection[0]['label'])

#print("emotion", overall_collection)
#display(overall_collection)

texttest['Emotion'] = overall_collection
display(texttest)

,Public_Opinion_Vaccine,label,score,Emotion
0,Making appointments,POSITIVE,0.996486,anger
1,If we were allowed to be exempt from classes f...,NEGATIVE,0.997259,sadness
2,A streamlined clinic that makes getting the va...,POSITIVE,0.998895,joy
3,Email updates about eligibility,POSITIVE,0.994933,joy
4,It being distributed at hours early and later ...,POSITIVE,0.995604,joy
...,...,...,...,...
1687,has effects on fertility,NEGATIVE,0.984783,joy
1688,The side effects.,NEGATIVE,0.996639,sadness
1689,People literally dying,NEGATIVE,0.995031,sadness
1690,I have allergies so I am afraid I might have a...,NEGATIVE,0.998603,fear


In [40]:
#texttest.to_excel("Sentiment_Emotion_Results_score.xlsx") 